In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import tqdm

train_df=pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")
test_df=pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")

In [ ]:
#import pandas_profiling as pdp #This pandas_profiling library can build Report on a given dataframe
#pdp.ProfileReport(train_df)


In [ ]:
#plt.figu
y=train_df.iloc[1:,1]
x=range(0,np.size(y))
plt.figure(figsize=(16,2),tight_layout=True)
plt.plot(x,y)
set1={'family':'bold','color':'blue','size':20}
plt.title("Degree In Celcius",fontdict=set1)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(16,40),tight_layout=True)
for i in range(1,len(train_df.columns)):
    plt.subplot(11,1,i)#subplot(total subplots,rrow_no,col_no)
    plt.title(train_df.columns[i],fontdict=set1)
    plt.plot(train_df.iloc[1:,i])
    plt.grid()
plt.show()


In [ ]:
all_df=pd.concat([train_df,test_df]).reset_index(drop=True)
plt.figure(figsize=(16,40),tight_layout=True)
for i in range(1,9):
    plt.subplot(11,1,i)
    plt.title(all_df.columns[i],fontdict=set1)
    plt.plot(all_df.iloc[1:train_df.shape[0],i])
    plt.plot(all_df.iloc[train_df.shape[0]:,i])
    
    #plt.plot(all_df[i][1:train_df.shape[0]])
    #plt.plot(all_df[i][train_df.shape[0]:])
    plt.grid()
plt.show()


In [ ]:
features=[feature for feature in train_df.columns if feature not in ["date_time","target_carbon_monoxide", 
                                                                       "target_benzene", "target_nitrogen_oxides"]]





In [ ]:
fig=plt.figure(figsize=(16,10))
upd_fig=fig.add_gridspec(3,3)
upd_fig.update(wspace=-.2,hspace=0.25)


In [ ]:
i=1
set1={'family':'bold','color':'blue','size':20}
fig=plt.figure(figsize=(16,20),tight_layout=True)
for col in features:
    plt.subplot(8,1,i)
    sns.kdeplot(train_df[col],shade=True, color='#fcd12a', alpha=0.85)#Here alpha is actually opacity of graph
    sns.kdeplot(test_df[col],shade=True, color='#287094', alpha=0.85)#if we keep writing this
    plt.title(col,fontdict=set1)
    plt.grid()
    i=i+1                                                           #we can draw multiple plot in single graph


plt.show()



   

In [ ]:
features

In [ ]:
X=train_df[features]

In [ ]:
Y=train_df[['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)

In [ ]:
import tensorflow as tf
import keras

In [ ]:
model=tf.keras.Sequential([tf.keras.layers.Dense(units=8,input_dim=8,activation='relu'),
                           tf.keras.layers.Dense(units=10,activation='relu'),
                           tf.keras.layers.Dense(units=16,activation='relu'),
                           tf.keras.layers.Dense(units=5,activation='relu'),
                           tf.keras.layers.Dense(3,activation='relu')])
model.compile(optimizer ='adam',loss  =tf.keras.losses.MeanSquaredLogarithmicError(),metrics=['mse'])
model.summary()#try to use AdaDelta optimizer in the next run

In [ ]:
history=model.fit(x_train,y_train,epochs=1000,validation_data=(x_test,y_test))

In [ ]:
epochs=1001
plt.figure(figsize = (12,10))
plt.subplot(2,1,1)
plt.plot(range(1,epochs),history.history['loss'])
plt.plot(range(1,epochs),history.history['val_loss'])
plt.title('Validation and Train loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.subplot(2,1,2)
plt.plot(range(1,epochs),history.history['mse'])
plt.plot(range(1,epochs),history.history['val_mse'])
plt.title('Validation and Train mse')
plt.xlabel('epochs')
plt.ylabel('mse')
plt.grid()

In [ ]:
sub=pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")
sub.columns

In [ ]:
test = test_df.drop(columns = ['date_time'])

In [ ]:
prediction=model.predict(test)

In [ ]:
predictions=pd.DataFrame(prediction)

In [ ]:
sub['target_carbon_monoxide']=predictions[0]
sub['target_benzene']=predictions[1]
sub['target_nitrogen_oxides']=predictions[2]


In [ ]:
sub.to_csv('submission.csv',index=False)